In [1]:
'''
手机抓包
'''

import re , urllib , http.cookiejar , base64 , binascii , rsa, json, time, math, os
import sqlite3
from log import log
import codecs  
from datetime import date



__version__ = '0.0.1'
__author__ = 'Pidong Li (hope-dream@163.com)'
cj = http.cookiejar.LWPCookieJar()
cookie_support = urllib.request.HTTPCookieProcessor(cj)
opener = urllib.request.build_opener(cookie_support , urllib.request.HTTPHandler)
urllib.request.install_opener(opener)
con = sqlite3.connect('cuiyuan.db')
cu = con.cursor()


In [ ]:
headers = {
"User-Agent":" Dalvik/1.6.0 (Linux; U; Android 4.1.1; MI 2A MIUI/4.12.5)",
"Host":" cy.zhubaotown.com",
"Connection":" Keep-Alive",
"Accept-Encoding":" gzip",
"Content-Type":" application/x-www-form-urlencoded"
}

In [ ]:
# login
def get_token():
    '''
    发送账号和密码（加密过的），获得token
    '''
    para_login = 'bundleId=com.zbxz.cuiyuan&onlyCode=91078cb040f2e105465919fa0e08facd&passWord=MTIzNDU2&sysType=android&userName=13679206003&uId=16891&bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1'
    url_login = 'http://cy.zhubaotown.com/index.php/user/pwdLogin/'
    data = para_login.encode('utf-8')
    request = urllib.request.Request(url_login , data, headers)
    response = urllib.request.urlopen(request)
    text = response.read().decode('utf-8')
    text
    token = json.loads(text)['userInfo']['token']
    return token

token = get_token()

In [ ]:
def get_shop_list(page, size):

    '''
        获得商家列表，以json导出， 城市选择四会，screenRegionId=65
    '''

    para_shop_list = 'orderKeyword=desc&orderValue=sort_order&page=%d&screenRegionId=65&size=%d&uId=16891&token=%s\
    &bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (page, size, token)
    url_shop_list = 'http://cy.zhubaotown.com/index.php/shop/allShopList/'
    data = para_shop_list.encode('utf-8')

    request = urllib.request.Request(url_shop_list , data, headers)
    try:
        response = urllib.request.urlopen(request)
        text = response.read().decode('utf-8')
        
        code = json.loads(text)['code']
        if code == 409:
            print('token错误，请重新检查')
            raise SystemExit
        if code == 414: # token出现错误需要重新检查
            print('No matching data')
            return 1
        if not code == 200:
            print('code:' + str(code) + json.loads(text)['prompt'])
            return 0
        out = json.loads(text)['datas']
    except Exception as e:
        print(e)
        out = 0;
    return out


In [ ]:
token = get_token()


In [ ]:
# 商家列表 shop_list 写入数据库，不清楚具体的商铺数目，先试10000个
def db_shop_list(update = True):
    table_name = 'shop_list'
    if update:
        table_name = 'shop_list_new'
    cu.execute('create table if not exists %s(shopId int, shopLogo varchar(255), shopMaster varchar(100), \
    shopName varchar(255), shopSn varchar(255), shopTotalGoods int)' %table_name)
    con.commit()
    shops = []
    print('正在更新商铺列表')
    for ii in range(100):
        try:
            print('商铺列表第%d页/每页100家……' % (ii+1), end = '\r')
            shops = get_shop_list(ii+1, 100)
            if shops == 1:
                print('商铺列表加载完成……')
                break
            if not shops == 0:
                for shop in shops:
                    cu.execute(r"insert into %s values(?, ?, ?, ?, ?, ?)" % table_name, (shop['shopId'], shop['shopLogo'], shop['shopMaster'], shop['shopName'], shop['shopSn'], shop['shopTotalGoods']))
                con.commit()
        except Exception as e:
            print(e)
            con.rollback()


In [ ]:
def get_shop_list_num(update = True):
    table_name = 'shop_list'
    if update:
        table_name = 'shop_list_new'
    cu.execute('select distinct shopId, shopTotalGoods from %s' % table_name)
    shop_ids_nums = cu.fetchall()
    return shop_ids_nums # 包含 商铺的id和拥有物品的数目

In [ ]:
# 根据商铺数据，得到物品的id，这是一页的
def shop2goods_id_page(page, shop_id, size):
    para_goods_id = 'goodsNew=0&isRecommended=0&page=%d&shopId=%d&size=%s&uId=16891&token=%s\
    &bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (page ,shop_id,  size, token)
    url_goods_id = 'http://cy.zhubaotown.com/index.php/shop/shopGoodsList_new/'
    data = para_goods_id.encode('utf-8')
    try:
        request = urllib.request.Request(url_goods_id , data, headers)
        response = urllib.request.urlopen(request)
        text = response.read().decode('utf-8')
        
        code = json.loads(text)['code']
        if code == 409:
            print('token错误，请重新检查')
            raise SystemExit

        if not code == 200:
            print('code:' + str(code) + json.loads(text)['prompt'])
            return 0
        xx = json.loads(text)['datas']
    except Exception as e:
        xx = 0
    return(xx)

In [ ]:
# 根据商铺数据，得到全部物品的id，
def shop2goods_id(shop_id, shopTotalGoods, size = 100):
# 获得商家的物品
    goods = []
    if shopTotalGoods>100 :
        for page in range(math.floor(shopTotalGoods/size)+1):
            
            xx = shop2goods_id_page(page + 1, shop_id, size)
            goods = goods + xx
    elif 0< shopTotalGoods <101:
        xx = shop2goods_id_page(1, shop_id, shopTotalGoods)
        goods = goods + xx
    else:
        print('chucuole')
    return goods

In [ ]:
shop2goods_id(1160, 10)

In [ ]:
# shop_ids2goods_ids 写入数据库
#新建数据库 shop2goods
def db_shop2goods(update = True):
    table_name = 'shop2goods'
    if update:
        table_name = 'shop2goods_new'

    cu.execute('create table if not exists %s (shop_id int , goods_id int, date varchar(100))' % table_name)
    con.commit()
    index = 0
    shop_ids_nums = get_shop_list_num(update)
#     print(len(shop_ids_nums))
    print('正在更新商家-物品的联系')
    for shop_ids_num in shop_ids_nums:
        index = index+1
        
        print('已完成' + str(round(index/len(shop_ids_nums)*100, 3)) + '%,请耐心等待……', end = '\r')
        try:
        
            goods_list = shop2goods_id(shop_ids_num[0], shop_ids_num[1])
            for goods in goods_list:
                cu.execute("insert into %s values(%d, %d, '%s')" %(table_name, shop_ids_num[0], goods['goodsId'], goods['add_time']))
            con.commit()
        except Exception as e:
            print(e)
            con.rollback()

In [ ]:
# 获取 goods_info
def get_goods_info(goods_id):
    '''
    通过goods_id获取内容
    '''
    goods_id = str(goods_id)
    para_goods_info = 'goodsId=%s&uId=16891&token=%s&bundleId=com.zbxz.cuiyuan&product_type=cyretail&sysType=1' % (goods_id, token)
    url_goods_info = 'http://cy.zhubaotown.com/index.php/goods/getGoodsInfo/'
    data = para_goods_info.encode('utf-8')
    try:
        request = urllib.request.Request(url_goods_info , data, headers)
        response = urllib.request.urlopen(request)
        text = response.read().decode('utf-8')
        code = json.loads(text)['code']
        if code == 409:
            print('token错误，请重新检查')
            raise SystemExit

        if not code == 200:
#             print('code:' + str(code) + json.loads(text)['prompt'])
            return 0
        xx = json.loads(text)['goodsInfo']
        
    except Exception as e:
#         print(e)
        xx = 0
    return(xx)



In [ ]:
# 删除物品数据,重新建表
def db_manipulate():
#     cu.execute('drop table if exists goods_info')
#     cu.execute('drop table if exists goods_img_url')
    # 物品信息写入数据库,21列
    # 创建数据库
    cu.execute('create table if not exists goods_info (goodsId varchar(20), appointmentedStatus varchar(20), \
                goodsInfo varchar(200), shopId varchar(20), title varchar(255),\
                 goodsMaterialId varchar(20), goodsSn varchar(20), goodsTypeId varchar(20),\
                has_cert varchar(20),isAppointmented varchar(20), isConcern varchar(20), isRecommended varchar(20),isimplant varchar(20), orderStatus varchar(20),\
                price real, priceRangeName varchar(255), quantity varchar(20),\
                goodPattern varchar(100), goodPatternId varchar(20), goodsCateId int, goodsCateName varchar(100),\
                img_str text, img_str_1 text)')
    cu.execute('create table if not exists goods_info_add (goodsId varchar(20), appointmentedStatus varchar(20), \
                goodsInfo varchar(200), shopId varchar(20), title varchar(255),\
                 goodsMaterialId varchar(20), goodsSn varchar(20), goodsTypeId varchar(20),\
                has_cert varchar(20),isAppointmented varchar(20), isConcern varchar(20), isRecommended varchar(20),isimplant varchar(20), orderStatus varchar(20),\
                price real, priceRangeName varchar(255), quantity varchar(20),\
                goodPattern varchar(100), goodPatternId varchar(20), goodsCateId int, goodsCateName varchar(100),\
                img_str text, img_str_1 text)')
    cu.execute('create table if not exists goods_info_remove (goodsId varchar(20), appointmentedStatus varchar(20), \
                goodsInfo varchar(200), shopId varchar(20), title varchar(255),\
                 goodsMaterialId varchar(20), goodsSn varchar(20), goodsTypeId varchar(20),\
                has_cert varchar(20),isAppointmented varchar(20), isConcern varchar(20), isRecommended varchar(20),isimplant varchar(20), orderStatus varchar(20),\
                price real, priceRangeName varchar(255), quantity varchar(20),\
                goodPattern varchar(100), goodPatternId varchar(20), goodsCateId int, goodsCateName varchar(100))')
    cu.execute('create table if not exists goods_img_url (goods_id int, img_url text)')

    con.commit()

In [ ]:
db_manipulate()

In [ ]:
# 图片url写入数据库

def get_img_url(goods_id, img_arr):
    con.commit()
    img_str = ''
    img_str_1 = ''
    index = 0
    # 54bcde343ac35:1:0:|;55f13ae611bf1:1:1:|;55f13f9b3254c:1:2:|;
    for img in img_arr:
        
        path, file_name = os.path.split(img['imgUrl'])
        cu.execute('insert into goods_img_url values(?, ?)', (goods_id, img['imgUrl']) )
        
        img_str = img_str + r'<P><IMG src="file:///C:/图片/%s"></P>' % (file_name)
        img_str_1 = img_str_1 + os.path.splitext(file_name)[0] + ':1:' + str(index) + ':|;'
        index += 1
        
    return img_str, img_str_1
        



    
    

In [ ]:

logger = log('cuiyuan.log', 'goods_info')
def db_goods_info(goods_ids, status = None, append = False):
    index = 0
    goods_ids_len = len(goods_ids)
    if append:
        cu.execute('select distinct goodsId from goods_info')
        goods_id_remain = cu.fetchall()
        goods_id_remain = tuple2list(goods_id_remain)
        #取差集
        goods_ids = list(set(goods_ids).difference(set(goods_id_remain)))
        index = len(goods_id_remain)
        
    print('正在更新物品信息…………')
    for goods_id in goods_ids:
        index = index +1;
        print(r'已完成 '+ str( round(index/goods_ids_len*100,3)) + '%, 请耐心等待', end = '\r')
        try:
            goods_info = get_goods_info(goods_id)
            if not goods_info == 0:
                if status is None or status == 'add':
#                     pass
                    img_str, img_str_1 = get_img_url(goods_id, goods_info['goodsImgArr'])
                    cu.execute("insert into goods_info values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?, ?, ?)" , \
                               (goods_info['goodsId'], goods_info['appointmentedStatus'], goods_info['goodsInfo'], goods_info['shopId'], \
                                goods_info['title'], goods_info['goodsMaterialId']\
                               , goods_info['goodsSn'], goods_info['goodsTypeId'], goods_info['has_cert'], goods_info['isAppointmented'], goods_info['isConcern']\
                               , goods_info['isRecommended'], goods_info['isimplant'], goods_info['orderStatus'], goods_info['price'], goods_info['priceRangeName'], goods_info['quantity'], goods_info['goodPattern']\
                               , goods_info['goodPatternId'], goods_info['goodsCateId'], goods_info['goodsCateName'], img_str, img_str_1))

                    con.commit()
                if status == 'add':
                    cu.execute("insert into goods_info_add values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?, ?, ?)" , \
                               (goods_info['goodsId'], goods_info['appointmentedStatus'], goods_info['goodsInfo'], goods_info['shopId'], \
                                goods_info['title'], goods_info['goodsMaterialId']\
                               , goods_info['goodsSn'], goods_info['goodsTypeId'], goods_info['has_cert'], goods_info['isAppointmented'], goods_info['isConcern']\
                               , goods_info['isRecommended'], goods_info['isimplant'], goods_info['orderStatus'], goods_info['price'], goods_info['priceRangeName'], goods_info['quantity'], goods_info['goodPattern']\
                               , goods_info['goodPatternId'], goods_info['goodsCateId'], goods_info['goodsCateName'], img_str, img_str_1))

                    con.commit()
                if status == 'remove':
                    cu.execute("insert into goods_info_remove values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?, ?, ?)" , \
                               (goods_info['goodsId'], goods_info['appointmentedStatus'], goods_info['goodsInfo'], goods_info['shopId'], \
                                goods_info['title'], goods_info['goodsMaterialId']\
                               , goods_info['goodsSn'], goods_info['goodsTypeId'], goods_info['has_cert'], goods_info['isAppointmented'], goods_info['isConcern']\
                               , goods_info['isRecommended'], goods_info['isimplant'], goods_info['orderStatus'], goods_info['price'], goods_info['priceRangeName'], goods_info['quantity'], goods_info['goodPattern']\
                               , goods_info['goodPatternId'], goods_info['goodsCateId'], goods_info['goodsCateName']))
                    cu.execute('delete from goods_info where goodsId = ' + str(goods_id))
                    con.commit()
     
        except Exception as e:
            print(e)
            con.rollback()
    print('更新完成')

In [ ]:
cu.close()
con.close()

In [ ]:
# 将数据库返回值的变为list（仅是第一列）
def tuple2list(tuple_list):
    out = []
    for xx in tuple_list:
        out.append(str(xx[0]))
    return out

In [ ]:
# 原始数据收集
def init():
    db_shop_list(update = False) # 获得商铺列表
    db_shop2goods(update = False) # 获得物品列表
    
init()
    

In [ ]:
token = get_token()
cu.execute('select distinct goods_id from shop2goods')
goods_ids = tuple2list(cu.fetchall())
db_goods_info(goods_ids, append = True)

In [5]:
def db2csv(table_name, colnames, path):
    cu.execute('select distinct * from %s' %table_name)
    data = cu.fetchall()
    write_csv(data, colnames, path)
    
def write_csv(data, colnames, path):
    
    csvfile = open(path, 'wb')  
    csvfile.write(codecs.BOM_UTF8) # 二进制写入bom解决excel打开乱码问题
    csvfile.close()
    csvfile = open(path, 'a', encoding = 'utf-8')  
    writers = csv.writer(csvfile) 
    writers.writerow(colnames)
    writers.writerows(data)
    csvfile.close()

In [ ]:
'''
更新逻辑，检查商家的数据增减情况，然后根据最新的商家信息更新物品信息，物品信息与元数据匹配，
增加的与减少的分布建立新的数据库，同时对原数据库进行增减
'''

def update():
    '''
    比较商家是否增加或减少
    '''
    db_shop_list(update = True) # 获得最新商铺列表
    db_shop2goods(update = True) # 获得最新物品列表
    colnames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods']
    date_str = str(data.today())
    cu.execute('select * from shop_list_new where shopId not in (select shopId from shop_list)')
    shop_list_add = con.fetchall()
    if len(shop_list_add) > 0:
        print('商家增加了%d所，将以csv文件导出' %len(shop_list_add))
        
        db2csv(shop_list_add, colnames, '增加商家-%s.csv'% date_str)
    else:
        print('商家并未增加')
    cu.execute('select * from shop_list where shopId not in (select shopId from shop_list_new)')
    shop_list_remove = con.fetchall()
    if len(shop_list_remove) > 0:
        print('商家减少了%d所，将以csv文件导出' %len(shop_list_remove))
        db2csv(shop_list_remove, colnames, '减少商家-%s.csv'% date_str)
    else:
        print('商家并没有减少')
    
    # 比较物品的增删
    goods_info_add_colnames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods', \
           'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' ,\
                'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , \
                'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' ,\
                'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName', 'img_str', 'img_str_1']
    cu.execute('select goods_id from shop2goods_new where goods_id not in (select goods_id from shop2goods)')
    goods_add = cu.fetchall()
    if len(goods_add)>0 :
        print('物品增加了%d个，将以csv文件导出' %len(goods_add))
        goods_add_id = tuple2list(goods_add)
        db_goods_info(goods_add_id, status = 'add') # 将新加入的写入 goods_info 和 goods_info_add 数据库
        cu.execute('select * from shop_list_new inner join goods_info_add on shop_list_new.shopId = goods_info_add.shopId')
        shop_goods_add = cu.fetchall()
        
        write_csv(goods_info_add, goods_info_add_colnames, '增加物品-%s.csv'% date_str )
    else:
        print('物品并未增加')
    
    cu.execute('select goods_id from shop2goods where goods_id not in (select goods_id from shop2goods_new)')
    goods_add = cu.fetchall()
    goods_info_remove_colnames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods', \
           'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' ,\
                'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , \
                'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' ,\
                'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
    if len(goods_add)<0 :
        print('物品减少了%d个，将以csv文件导出' %len(goods_add))
        goods_add_id = tuple2list(goods_add)
        db_goods_info(goods_add_id, status = 'remove') # 将删除的写入 goods_info_remove 数据库，并从goods_info数据库中删除
        cu.execute('select * from shop_list inner join goods_info_remove on shop_list.shopId = goods_info_remove.shopId')
        shop_goods_remove = cu.fetchall()
        
        write_csv(goods_info_remove, goods_info_add_colname, '增加物品-%s.csv'% date_str )
    else:
        print('物品并未减少')
    # 导出数据
    
    

In [ ]:
def db_rename():
    '''
    将更新的数据表的名称修改为 普通表
    ALTER TABLE testtable RENAME TO testtable2;
    '''
    cu.execute('drop table shop_list')
    cu.execute('alter table shop_list_new rename to shop_list')
    cu.execute('drop table shop2goods')
    cu.execute('alter table shop2goods_new rename to shop_goods')
    cu.execute('drop table goods_info_add')
    cu.execute('drop table goods_info_remove')
    con.commit()

In [ ]:
cu.close()
con.close()

In [9]:
cu.execute('select * from goods_img_url')
img_url_all = cu.fetchall()

colnames = ['goodsId', 'goods_img_url']
write_csv(img_url_all, colnames, '物品图片.csv')


In [3]:
import csv

In [6]:

cu.execute('select * from shop_list inner join goods_info on shop_list.shopId = goods_info.shopId')
shop_goods_all = cu.fetchall()
colnames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods', \
           'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' ,\
            'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,\
            'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', \
            'goodPatternId' , 'goodsCateId', 'goodsCateName', 'img_str', 'img_str_1']

write_csv(shop_goods_all, colnames, '商家-物品信息.csv')

In [7]:
cu.execute('select * from shop_list')
data = cu.fetchall()
colnames = ['shopId', 'shopLogo' , 'shopMaster','shopName', 'shopSn', 'shopTotalGoods']
#            'goodsId' , 'appointmentedStatus' , 'goodsInfo' , 'shopId' , 'title' , 'goodsMaterialId' , 'goodsSn' , 'goodsTypeId' , 'has_cert' ,'isAppointmented' , 'isConcern' , 'isRecommended' ,'isimplant' , 'orderStatus' , 'price', 'priceRangeName' , 'quantity' , 'goodPattern', 'goodPatternId' , 'goodsCateId', 'goodsCateName']
write_csv(data, colnames, '商家列表.csv')